In [15]:
import numpy as np

class GeneticAlgorithm:
    def __init__(self, population_size, mutation_rate, crossover_rate, elitism_count):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.elitism_count = elitism_count
    
    def init_population(self, chromosome_len):
        population = Population(self.population_size, chromosome_len)
        return population

    def calculate_fitness(self, individual):
        '''
        count the number of ones in the chromosome, and then normalize
        the output to be between zero and one by dividing by the
        chromosome length
        '''
        correct_genes = 0
        for gene_index in range(individual.chromosome_len):
            if int(individual.get_gene(gene_index)) is 1:
                correct_genes += 1
                
        # Calculate fitness
        fitness = correct_genes / individual.chromosome_len
        individual.fitness = fitness
        return fitness
    
    def eval_population(self, population):
        '''
        loop over every individual in the population and evaluate them
        '''
        population_fitness = 0
        
        for i, individual in enumerate(population.individuals):
            population_fitness += self.calculate_fitness(individual)
        
        population.population_fitness = population_fitness
    
    def is_termination_condition_met(self, population):
        for i, individual in enumerate(population.individuals):
            if int(individual.fitness) is 1:
                return True
        return False
    
    def select_parent(self, population):
        # Get individuals
        individuals = population.individuals
        
        # Spin roulette wheel
        population_fitness = population.population_fitness
        roulette_wheel_position = random.random() * population_fitness
        
        # Find parent
        spin_wheel = 0
        for individual in individuals:
            spin_wheel += individual.fitness
            if spin_wheel >= roulette_wheel_position:
                return individual
        
        # Return the last one if none of them matched
        return individuals[-1]
    
    def crossover_population(self, population, chromosome_len):
        # Create new population
        new_population = Population(population.size, chromosome_len)
        
        for population_index in range(population.size):
            parent1 = population.get_fittest(population_index)
            # Can we apply crossover to this individual?
            if self.crossover_rate > random.random() and population_index > self.elitism_count:
                # Initialze offspring
                offspring = Individual(parent1.chromosome_len)
                
                # Find second parent
                parent2 = self.select_parent(population)

                # Loop over genome
                for gene_index in range(parent1.chromosome_len):
                    # Use half's the parent1's gene and half of parent2's gene
                    if random.random() > 0.5:
                        offspring.set_gene(gene_index, parent1.get_gene(gene_index))
                    else:
                        offspring.set_gene(gene_index, parent2.get_gene(gene_index))
                # Add offspring to new population
                new_population.set_individual(population_index, offspring)
            else:
                # Add individual to new population without applying crossover
                new_population.set_individual(population_index, parent1)
        return new_population

    def mutate_population(self, population, chromosome_len):
        new_population = Population(self.population_size, chromosome_len)
        
        for population_index in range(population.size):
            individual = population.get_fittest(population_index)
            
            # Loop over individual's gene
            for gene_index in range(individual.chromosome_len):
                # Skip mutation if this is an elite individual
                if population_index > self.elitism_count:
                    # Does this gene need mutation?
                    if self.mutation_rate > random.random():
                        # Get new gene
                        new_gene = 1
                        if individual.get_gene(gene_index) == 1:
                            new_gene = 0
                        # Mutate gene
                        individual.set_gene(gene_index, new_gene)
            # Add individual to population
            new_population.set_individual(population_index, individual)
        # Mutated population
        return new_population

In [4]:
class Individual:
    def __init__(self, chromosome_len):
        # Creates an array of random zeros and ones with a fixed len
        self._chromosome = np.random.randint(2, size=chromosome_len)
        self._fitness = -1
        pass
    
    @property
    def chromosome(self):
        return self._chromosome
    
    @property
    def chromosome_len(self):
        return len(self._chromosome)
    
    def get_gene(self, offset):
        return self._chromosome[offset]
    
    def set_gene(self, offset, gene):
        self._chromosome[offset] = gene
    
    @property 
    def fitness(self):
        return self._fitness
    
    @fitness.setter
    def fitness(self, fitness):
        self._fitness = fitness
    
    def __str__(self):
        output = ''
        for i, v in enumerate(self.chromosome):
            output += str(v)
        return output


In [9]:
from operator import attrgetter
import random

class Population:
    def __init__(self, population_size, chromosome_len):
        self._population_fitness = -1
        self._population = [Individual] * population_size

        for i in range(population_size):
            self._population[i] = Individual(chromosome_len)
    
    @property
    def individuals(self):
        return self._population
    
    def get_fittest(self, offset):
        # Sort by fitness
        self._population = sorted(self._population, key=attrgetter('fitness'), reverse=True)
        return self._population[offset]

    @property
    def population_fitness(self):
        return self._population_fitness

    @population_fitness.setter
    def population_fitness(self, fitness):
        self._population_fitness = fitness
    

    @property
    def size(self):
        return len(self._population)

    def set_individual(self, offset, individual):
        self._population[offset] = individual
    
    def get_individual(self, offset):
        return self._population[offset]

    def shuffle(self):
        random.shuffle(self._population)
        
population = Population(3, 3)
print(population.individuals[0])
print(population.get_fittest(1))
population.shuffle()

001
100


In [17]:
# Main class to initialize the Genetic Algorithm
class AllOnesGA:
    def __init__(self):
        ga = GeneticAlgorithm(100, 0.001, 0.95, 2)
        chromosome_len = 50
        
        population = ga.init_population(chromosome_len)
        generation = 1
        while not ga.is_termination_condition_met(population):
            print('best solution: {}'.format(population.get_fittest(0)))
            
            # Apply crossover
            population = ga.crossover_population(population, chromosome_len)
            
            # Apply mutation
            population = ga.mutate_population(population, chromosome_len)
            
            # Evaluate population
            ga.eval_population(population)
            
            # Increment the current generation
            generation += 1
        
        print('found solution in {}'.format(generation))
        print()
        print('best solution: {}'.format(population.get_fittest(0)))

all_ones = AllOnesGA()

best solution: 01101001100111000000000110110111101100100101000000
best solution: 00101111101110110101001010111101111111110011100010
best solution: 00111111111110100101001010110111111101110011100101
best solution: 01001111001110110101011101111100111100011111111011
best solution: 01001111001110110101011101111100111100011111111011
best solution: 01001111001110110101011101111100111100011111111011
best solution: 00101111011110111011001001111101111111101111110110
best solution: 00101111011110111011001001111101111111101111110110
best solution: 00101111011110111011001001111101111111101111110110
best solution: 00101111011110111011001001111101111111101111110110
best solution: 00101111011110111011001001111101111111101111110110
best solution: 00101111011110111011001001111101111111101111110110
best solution: 00101111011110111011001001111101111111101111110110
best solution: 00111111111110110101001110111101111101111011101110
best solution: 00111111111110110101001110111101111101111011101110
best solut